# Ansamble prediction from logits model 

# Read the data

In [1]:
import numpy  as np
import pandas as pd

In [2]:
opt_df_train1 = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Large_M2_XLNet_Base_train.csv")
opt_df_test1  = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Large_M2_XLNet_Base_test.csv")
opt_df_dev1   = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Large_M2_XLNet_Base_validation.csv")

opt_df_train2 = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Base_M2_Roberta_Base_train.csv")
opt_df_test2  = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Base_M2_Roberta_Base_test.csv")
opt_df_dev2   = pd.read_csv("../../data/optimism-twitter-data/processed/optimism_Ansamble_M1_Bert_Base_M2_Roberta_Base_validation.csv")

In [3]:
# #X_train
# np.vstack([opt_df_train["M1:Bert_Large_logit0"]\
#           , opt_df_train["M1:Bert_Large_logit1"]\
#           , opt_df_train["M2:XLNet_Base_logit0"]\
#           , opt_df_train["M2:XLNet_Base_logit1"]])#.shape
# #y_train
# np.where(opt_df_train["AverageAnnotation"]<=0, 0, 1)#.shape

opt_df_train1.head()

,Unnamed: 0,Tweet,Username,AverageAnnotation,Original ID,M1:Bert_Large_logit0,M1:Bert_Large_logit1,M2:XLNet_Base_logit0,M2:XLNet_Base_logit1
0,0,@runge_kelly rest is good!! if you like tea an...,TheAnglophiler,1.25,220,-5.205334,4.681307,-3.560899,3.339703
1,1,dr. budiani-saberi is a medical anthropologist...,JessiKersi,0.00,916,2.511252,-2.607783,2.411943,-2.060660
2,2,i want city to win. chelsea going up too far w...,Ajinkyaworld,1.50,321,-5.192333,4.742418,-2.866381,3.463603
3,3,you're annoying the piss out of me,TehhKota,-1.80,2127,4.497906,-4.169426,4.160317,-3.558687
4,4,@arapahoe_basin looking good for tomorrow's sk...,tomfricke,0.60,5643,-5.039854,4.488212,-3.775995,3.775713


In [7]:
#XGBoost
#https://scikit-learn.org/stable/modules/ensemble.html#adaboost

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn          import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# X, y = make_hastie_10_2(random_state=0)
# X_train, X_test = X[:2000], X[2000:]
# y_train, y_test = y[:2000], y[2000:]

X_train = np.vstack([opt_df_train1["M1:Bert_Large_logit0"]\
          , opt_df_train1["M1:Bert_Large_logit1"]\
          , opt_df_train1["M2:XLNet_Base_logit0"]\
          , opt_df_train1["M2:XLNet_Base_logit1"]\
          , opt_df_train2["M1:Bert_Base_logit0"]\
          , opt_df_train2["M1:Bert_Base_logit1"]\
          , opt_df_train2["M2:Roberta_Base_logit0"]\
          , opt_df_train2["M2:Roberta_Base_logit1"]]).T
y_train = np.where(opt_df_train1["AverageAnnotation"]<=0, 0, 1)

X_dev = np.vstack([opt_df_dev1["M1:Bert_Large_logit0"]\
          , opt_df_dev1["M1:Bert_Large_logit1"]\
          , opt_df_dev1["M2:XLNet_Base_logit0"]\
          , opt_df_dev1["M2:XLNet_Base_logit1"]\
          , opt_df_dev2["M1:Bert_Base_logit0"]\
          , opt_df_dev2["M1:Bert_Base_logit1"]\
          , opt_df_dev2["M2:Roberta_Base_logit0"]\
          , opt_df_dev2["M2:Roberta_Base_logit1"]]).T
y_dev = np.where(opt_df_dev1["AverageAnnotation"]<=0, 0, 1)

X_test = np.vstack([opt_df_test1["M1:Bert_Large_logit0"]\
          , opt_df_test1["M1:Bert_Large_logit1"]\
          , opt_df_test1["M2:XLNet_Base_logit0"]\
          , opt_df_test1["M2:XLNet_Base_logit1"]\
          , opt_df_test2["M1:Bert_Base_logit0"]\
          , opt_df_test2["M1:Bert_Base_logit1"]\
          , opt_df_test2["M2:Roberta_Base_logit0"]\
          , opt_df_test2["M2:Roberta_Base_logit1"]]).T
y_test = np.where(opt_df_test1["AverageAnnotation"]<=0, 0, 1)

clf1  = HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)
clf2  = svm.NuSVC(gamma='auto').fit(X_train, y_train)#.fit(X, Y)
clf3  = MLPClassifier(hidden_layer_sizes=(32, 16, 4), random_state=1).fit(X_train, y_train)
clf4  = KNeighborsClassifier(n_neighbors=20).fit(X_train, y_train)

print("XGBoost: \nTrain:      %.4f, \nValidation: %.4f, \nTest:       %.4f;"\
      %(clf1.score(X_train, y_train).round(4)\
      , clf1.score(X_dev, y_dev).round(4)\
      , clf1.score(X_test, y_test).round(4)))
print()
print("SVM: \nTrain:      %.4f, \nValidation: %.4f, \nTest:       %.4f;   "\
      %(clf2.score(X_train, y_train).round(4)\
        , clf2.score(X_dev, y_dev).round(4)\
        , clf2.score(X_test, y_test).round(4)))
print()
print("MLP: \nTrain:      %.4f, \nValidation: %.4f, \nTest:       %.4f;   "\
      %(clf3.score(X_train, y_train).round(4)\
        , clf3.score(X_dev, y_dev).round(4)\
        , clf3.score(X_test, y_test).round(4)))
print()
print("KNN: \nTrain:      %.4f, \nValidation: %.4f, \nTest:       %.4f.   "\
      %(clf4.score(X_train, y_train).round(4)\
      , clf4.score(X_dev, y_dev).round(4)\
      , clf4.score(X_test, y_test).round(4)))

XGBoost: 
Train:      1.0000, 
Validation: 0.8177, 
Test:       0.8268;

SVM: 
Train:      0.9951, 
Validation: 0.8346, 
Test:       0.8190;   

MLP: 
Train:      0.9992, 
Validation: 0.8255, 
Test:       0.8242;   

KNN: 
Train:      0.9988, 
Validation: 0.8281, 
Test:       0.8320.   


In [20]:
X_train.shape, X_test.shape

((4, 5939), (4, 768))